Загрузите данные


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName("Spark").getOrCreate()


In [2]:
from pyspark.sql.types import *

data_shema = [StructField("sepal_length",DoubleType(),True), StructField("sepal_width",DoubleType(),True), StructField("petal_length",DoubleType(),True), StructField("petal_width",DoubleType(),True), StructField("variety",StringType(),True)]
final_struc = StructType(fields = data_shema)
df = spark.read.csv("iris.csv",sep=',',header=True, schema=final_struc)
df.show()

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
|         5.4|        3.9|         1.7|        0.4| Setosa|
|         4.6|        3.4|         1.4|        0.3| Setosa|
|         5.0|        3.4|         1.5|        0.2| Setosa|
|         4.4|        2.9|         1.4|        0.2| Setosa|
|         4.9|        3.1|         1.5|        0.1| Setosa|
|         5.4|        3.7|         1.5|        0.2| Setosa|
|         4.8|        3.4|         1.6|        0.2| Setosa|
|         4.8|        3.0|         1.4|        0.1| Setosa|
|         4.3|        3.0|         1.1| 

In [8]:
#При помощи VectorAssembler преобразовать все колонки с признаками в одну (использовать PipeLine)
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

ignore = ['variety']
assembler = VectorAssembler(
    inputCols=[x for x in df.columns if x not in ignore],
    outputCol='features')

#assembler.transform(df)

#Convert to numeric
qualification_indexer = StringIndexer(inputCol="variety", outputCol="varietyIndex")

#Create pipeline and pass it to stages
pipeline = Pipeline(stages=[
           qualification_indexer, 
           assembler
])
#fit and transform
df_transformed = pipeline.fit(df).transform(df)
df_transformed.show()

+------------+-----------+------------+-----------+-------+------------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|varietyIndex|         features|
+------------+-----------+------------+-----------+-------+------------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|         0.0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|         0.0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|         0.0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|         0.0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|         0.0|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| Setosa|         0.0|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3| Setosa|         0.0|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2| Setosa|         0.0|[5.0,3.4,1.5,0.2]|

In [15]:
#Разбить данные на train и test
train, test = df_transformed.randomSplit([0.7, 0.3])

In [16]:
#Создать модель логистической регресии и обучить ее
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol='features',
                        labelCol='varietyIndex')
model = lr.fit(train)

In [21]:
#Воспользоваться MulticlassClassificationEvaluator для оценки качества на train и test множестве
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
predictions = model.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol='varietyIndex')
print('Evaluation:', evaluator.evaluate(predictions))

Evaluation: 0.9550406020994258
